In [2]:
import cv2
import os
import random
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
#Setting up paths 
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

LFW_PATH = 'lfw-deepfunneled'

#Creating folder structures
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)

In [ ]:
#Fetching negatives (LFW Dataset)
!tar -xf lfw-deepfunneled.tar

In [ ]:
#Move LFW Images to Negatives 
try:
  for directory in os.listdir(LFW_PATH):
    for filee in os.listdir(os.path.join(LFW_PATH, directory, filee)):
      EX_PATH = os.path.join(LFW_PATH, directory, filee)
      NEW_PATH = os.path.join(NEG_PATH, filee)
      os.replace(EX_PATH, NEW_PATH)

    print(f"Successfully moved LFW images to {NEG_PATH}.")
except:
  print("An error occured!")

In [ ]:
#Collect positive and anchor classes using webcam

#Establish connection to webcam
cap = cv2.VideoCapture(5)

while cap.isOpened():
  "Read results"
  ret, frame = cap.read()
  #Show image back to screen
  cv2.imshow('Image Collection', frame)
  #Breaking if 'q' key is pressed
  if cv2.waitKey(1) & 0XFF == ord('q'):
    break

#Release webcam
cap.release()
#Close .imshow() frame
cv2.destroyAllWindows